In [4]:
import pandas as pd
import numpy as np
import requests
import sqlite3
import datetime
import re

In [5]:
desired_columns = ["ListingID", "SubType", "MLSArea", "YrBuilt", "AcLSqft", "ViewYN", "PoolPrivateYN", "BAC"]

In [6]:
single_fam_homes = pd.read_csv("data/MLS_SINGLEFAMILYHOMES.csv")[desired_columns]
not_sfh = pd.read_csv("data/MLS_NOT_SFH.csv")[desired_columns]
new_mls_data = pd.read_csv("data/NEWER_MLS_DATA.csv")[desired_columns]

massage features such as listing ID to match other database & parse Acre/LotSquareFootage to only have LotSquareFootage

In [7]:
clean_mls = lambda x : x[2:]

single_fam_homes.ListingID = single_fam_homes.ListingID.apply(clean_mls)
not_sfh.ListingID = not_sfh.ListingID.apply(clean_mls)
new_mls_data.ListingID = new_mls_data.ListingID.apply(clean_mls)

In [8]:
clean_aclsqft = lambda x : int(x.split("/")[-1].replace(",", ""))

single_fam_homes.AcLSqft = single_fam_homes.AcLSqft.apply(clean_aclsqft)
not_sfh.AcLSqft = not_sfh.AcLSqft.apply(clean_aclsqft)
new_mls_data.AcLSqft = new_mls_data.AcLSqft.apply(clean_aclsqft)

1 hot encode features such as notable View & private pool on property

In [9]:
one_hot_view = lambda x : 1 if x == "Y" else 0
single_fam_homes.ViewYN = single_fam_homes.ViewYN.apply(one_hot_view)
not_sfh.ViewYN = not_sfh.ViewYN.apply(one_hot_view)
new_mls_data.ViewYN = new_mls_data.ViewYN.apply(one_hot_view)

In [10]:
one_hot_pool = lambda x : 1 if x == "Y" else 0
single_fam_homes.PoolPrivateYN = single_fam_homes.PoolPrivateYN.apply(one_hot_pool)
not_sfh.PoolPrivateYN = not_sfh.PoolPrivateYN.apply(one_hot_pool)
new_mls_data.PoolPrivateYN = new_mls_data.PoolPrivateYN.apply(one_hot_pool)

some years have "/EST" suffix

In [11]:
def parse_yrBuilt(x):
    try:
        return int(x.split("/")[0])
    except:
        return x

In [12]:
single_fam_homes.YrBuilt = single_fam_homes.YrBuilt.apply(parse_yrBuilt)
not_sfh.YrBuilt = not_sfh.YrBuilt.apply(parse_yrBuilt)
new_mls_data.YrBuilt = new_mls_data.YrBuilt.apply(parse_yrBuilt)

In [13]:
pattern = re.compile("([\d.]+)")
clean_bac = lambda x : re.search(pattern, x).group(0)

single_fam_homes.BAC = single_fam_homes.BAC.apply(clean_bac)
not_sfh.BAC = not_sfh.BAC.apply(clean_bac)
new_mls_data.BAC = new_mls_data.BAC.apply(clean_bac)

In [14]:
from sqlite_api import MLSDatabase

In [15]:
db = MLSDatabase()

In [16]:
db.create_db()

Opened database successfully
NOTICE:  table MLS_LISTINGS already exists
Table created successfully


In [17]:
db.insert_dataframe(single_fam_homes)
db.insert_dataframe(not_sfh)

Connected to slo_housing.db
Records created successfully
Connected to slo_housing.db
Records created successfully


In [18]:
from sqlite_api import SLOHouseDatabase

In [19]:
dbquery = SLOHouseDatabase()

In [21]:
len(dbquery.select_row("SELECT * FROM HOUSES JOIN MLS_LISTINGS USING (MLS_ID)"))

458